In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [47]:
X = pd.read_csv("./DeteccionDigitos/digit-recognizer/train.csv", sep=",")
Y = pd.read_csv("./DeteccionDigitos/digit-recognizer/test.csv", sep=",")

sample_submission = pd.read_csv("./DeteccionDigitos/digit-recognizer/sample_submission.csv", sep=",")

In [48]:
X.head(5)
# (42000, 785)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
Y.head(5)
# (28000, 784)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
label = X['label']
label

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X, X['label'], test_size=0.2, random_state=42)

Y_train = LabelBinarizer().fit_transform(Y_train)
Y_test = LabelBinarizer().fit_transform(Y_test)

X_train = X_train.drop(columns=['label'])
X_test = X_test.drop(columns=['label'])

print("Train X dataset shape: ", np.shape(X_train))
print("Train y dataset shape: ", np.shape(Y_train))

print("Test X dataset shape: ", np.shape(X_test))
print("Test y dataset shape: ", np.shape(Y_test))

Train X dataset shape:  (33600, 784)
Train y dataset shape:  (33600, 10)
Test X dataset shape:  (8400, 784)
Test y dataset shape:  (8400, 10)


In [52]:
# Define Sequential model with 2 layers
def create_classifier():
    input = Input(shape=(784,))

    x = Dense(500, activation='sigmoid')(input) # Input * (2/3)

    output = Dense(10, activation='softmax')(x)

    return Model(input, output)


model = create_classifier()
model.summary()


model.compile(loss='mse', metrics=['accuracy'], optimizer = Adam())

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_10 (Dense)            (None, 500)               392500    
                                                                 
 dense_11 (Dense)            (None, 10)                5010      
                                                                 
Total params: 397510 (1.52 MB)
Trainable params: 397510 (1.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=500, validation_data=(X_test, Y_test), verbose=1)

Epoch 1/100
68/68 [==============================] - 1s 10ms/step - loss: 0.0284 - accuracy: 0.8096 - val_loss: 0.0156 - val_accuracy: 0.9049
Epoch 2/100
68/68 [==============================] - 1s 9ms/step - loss: 0.0133 - accuracy: 0.9204 - val_loss: 0.0125 - val_accuracy: 0.9243
Epoch 3/100
68/68 [==============================] - 1s 9ms/step - loss: 0.0111 - accuracy: 0.9352 - val_loss: 0.0112 - val_accuracy: 0.9313
Epoch 4/100
68/68 [==============================] - 1s 9ms/step - loss: 0.0100 - accuracy: 0.9420 - val_loss: 0.0102 - val_accuracy: 0.9398
Epoch 5/100
68/68 [==============================] - 1s 8ms/step - loss: 0.0091 - accuracy: 0.9474 - val_loss: 0.0099 - val_accuracy: 0.9385
Epoch 6/100
68/68 [==============================] - 1s 9ms/step - loss: 0.0085 - accuracy: 0.9511 - val_loss: 0.0096 - val_accuracy: 0.9411
Epoch 7/100
68/68 [==============================] - 1s 9ms/step - loss: 0.0081 - accuracy: 0.9528 - val_loss: 0.0088 - val_accuracy: 0.9443
Epoch 8/100


In [54]:
pred = model.predict(Y)

875/875 [==============================] - 1s 1ms/step


In [55]:
df = pd.DataFrame(columns=["ImageId","Label"])

df.Label   = [_.argmax() for _ in pred]
df.ImageId = df.index+1
df.to_csv("./DeteccionDigitos/digit-recognizer/submission.csv", index = False)

df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
